In [1]:
import altair as alt
import pandas as pd
import numpy as np
import warnings
#warnings.simplefilter(action='ignore', category="SettingWithCopyWarning")
 

In [2]:
df = pd.read_csv("../data/chicago_bears_coaching_history.csv")
df = df.query("Year >= 1950")

df = df[~df.Coach.eq("Matt Eberflus")]
df = df.reset_index()
df["Made Playoffs"] = ~df.Result.isnull()
df_agg = df.groupby("Coach", as_index=False).agg({"W":"sum", "L":"sum", "W-L%":"mean"}).round(2)

In [3]:
# have a bar char where bar has two an x start and end date and sort by 
#chart_title = ""
selector = alt.selection_multi(fields=['Coach'])


y_cfgs = {"shorthand":"Coach", "title": ""}
x_cfgs = {"shorthand":"Year:O", "title":"Year"}
# background etc.
primary_color = "#0B162A" 
# font etc.
secondary_color = "#c83803"


In [4]:
heat = alt.Chart(df, width=1200, height=800, title="Chicago Bears Coaching History").mark_rect()\
.encode(
    y=alt.Y(**y_cfgs, sort=alt.EncodingSortField(field='Year', order='descending')),
    x=alt.X(**x_cfgs),
     color=alt.Color("Made Playoffs", scale=alt.Scale(domain=[True, False], range=[secondary_color, "white"])),
    tooltip=[
        alt.Tooltip("Coach", title="Coach"), 
        "W-L%", 
        alt.Tooltip("Year", title="Year"), 
        alt.Tooltip("W", title="Total Wins"), 
        alt.Tooltip("L", title="Losses"), 
        alt.Tooltip("Result", title="Result"),
        alt.Tooltip("Offense", title="Off. Coord."),
        alt.Tooltip("Defense", title="Def. Coord."),

        ]
).transform_filter(
    selector
)

In [5]:
chart_title2 = "Career Winning %"
y2_cfgs = {"shorthand":"Coach", "title": "Coach"}
x2_cfgs = {"shorthand":"W-L%", "title":"W-L%"}

bars = alt.Chart(df_agg, width=400, height=800, title="Career Winning%")\
.mark_bar(size=30, opacity=1, color="white")\
.encode(
    y=alt.Y(**y2_cfgs, sort=alt.EncodingSortField(field="W-L%", order="descending")),
    x=alt.X(**x2_cfgs),
    tooltip=[alt.Tooltip("Coach", title="Coach"),alt.Tooltip("W", title="Total Wins"), alt.Tooltip("L", title="Total Losses"), alt.Tooltip("W-L%", title="W-L%")]
).add_selection(selector).resolve_scale(y="independent")

In [6]:
(bars | heat).configure(background=primary_color)\
.configure_axis(gridColor="grey", gridDash=[6,4])\
.configure_axisLeft(labelColor=secondary_color, titleColor=secondary_color, labelFontSize=17, titleFontSize=20)\
.configure_axisBottom(labelColor=secondary_color, titleColor=secondary_color,labelFontSize=15, titleFontSize=17)\
.configure_legend(labelColor=secondary_color, titleColor=secondary_color, strokeColor='gray', fillColor=primary_color, padding=10,cornerRadius=10, labelFontSize=17, titleFontSize=17)\
.configure_title(color=secondary_color, fontSize=17)

alt.HConcatChart(...)